In [42]:
import os
import random

import numpy as np
from PIL import Image

In [247]:
width = 128
heigth = 128
channels = 1

def read_images(directory, k):
    images = []
    labels = []
    
    filenames = os.listdir(directory)
    random.shuffle(filenames)
    
    for filename in filenames[:k]:
        img = Image.open(os.path.join(directory, filename)).resize((width, heigth))
        images.append(np.array(img))
        labels.append(0 if 'cat' in filename else 1)
    return np.array(images, dtype=np.uint8), np.array(labels, dtype=np.uint8)

In [248]:
x, y = read_images('./data/train', 10000)

In [249]:
x = x / 255.0
x.shape

(10000, 128, 128, 3)

In [250]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout, Activation
from keras.callbacks import EarlyStopping

In [251]:
model = Sequential()

model.add(Conv2D(8, (3, 3), input_shape=x[0].shape, activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_210 (Conv2D)          (None, 128, 128, 8)       224       
_________________________________________________________________
max_pooling2d_194 (MaxPoolin (None, 64, 64, 8)         0         
_________________________________________________________________
conv2d_211 (Conv2D)          (None, 64, 64, 16)        1168      
_________________________________________________________________
max_pooling2d_195 (MaxPoolin (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_212 (Conv2D)          (None, 32, 32, 32)        4640      
_________________________________________________________________
max_pooling2d_196 (MaxPoolin (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_213 (Conv2D)          (None, 16, 16, 64)        18496     
__________

In [252]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)   
history = model.fit(
    x, y,
    batch_size=128,
    epochs=100, 
    validation_split=0.2,
    callbacks=[early_stopping]
)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 208s 26ms/step - loss: 0.6693 - acc: 0.5825 - val_loss: 0.6449 - val_acc: 0.6230
Epoch 2/100
8000/8000 [==============================] - 143s 18ms/step - loss: 0.6055 - acc: 0.6739 - val_loss: 0.5620 - val_acc: 0.7215
Epoch 3/100
8000/8000 [==============================] - 159s 20ms/step - loss: 0.5576 - acc: 0.7163 - val_loss: 0.5830 - val_acc: 0.6960
Epoch 4/100
8000/8000 [==============================] - 128s 16ms/step - loss: 0.5321 - acc: 0.7284 - val_loss: 0.5152 - val_acc: 0.7515
Epoch 5/100
8000/8000 [==============================] - 150s 19ms/step - loss: 0.5048 - acc: 0.7535 - val_loss: 0.4951 - val_acc: 0.7615
Epoch 6/100
8000/8000 [==============================] - 113s 14ms/step - loss: 0.4833 - acc: 0.7686 - val_loss: 0.4859 - val_acc: 0.7760
Epoch 7/100
8000/8000 [==============================] - 147s 18ms/step - loss: 0.4652 - acc: 0.7798 - val_loss: 0.4800 - va

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-252-f8fa4c75c215>", line 7, in <module>
    callbacks=[early_stopping]
  File "/usr/local/lib/python3.6/site-packages/keras/engine/training.py", line 1037, in fit
    validation_steps=validation_steps)
  File "/usr/local/lib/python3.6/site-packages/keras/engine/training_arrays.py", line 199, in fit_loop
    outs = f(ins_batch)
  File "/usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py", line 2666, in __call__
    return self._call(inputs)
  File "/usr/local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py", line 2636, in _call
    fetched = self._callable_fn(*array_vals)
  File "/usr/local/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1382, in __call__
    run_metadata_ptr)
KeyboardInterrupt

During handling o

KeyboardInterrupt: 

In [187]:
print(model.predict(x[:10]), y[:10])

[[0.01016274]
 [0.9897674 ]
 [0.00880679]
 [0.06050279]
 [0.8132624 ]
 [0.9962716 ]
 [0.11523642]
 [0.9724272 ]
 [0.8890431 ]
 [0.9999777 ]] [0 1 0 0 1 1 0 1 1 1]


In [70]:
x[0].shape

(64, 64, 3)